In [0]:
import requests
import json
from datetime import datetime
from datetime import timedelta
import os
import pandas as pd 
import time
import  pyspark.sql.functions as F
from pyspark.sql.types import *

In [0]:
def checkKeys(k,d):
  if k in d.keys():
    return True
  else:
    return False

In [0]:
#function to receive the data from API
def getAPIdata(pageNum):
  url='https://api.cyberclinic.com.au/api/admin/fetch_appointment_data?token=b8b3dbe3-e22d-4595-b78c-25d5a6f4e24b&page_size=100&page_number={}'  
  response = requests.get(url.format(pageNum), headers=None)
  return response

In [0]:
#code is calling the api, receiving the data and formetting the same 
df=pd.DataFrame()
for i in  range(1,100):
  res=getAPIdata(i)
  if res.status_code==200:
    if checkKeys('data',res.json()):
      tdf=pd.DataFrame(res.json()['data']['appointments'])
      df=pd.concat([df,tdf])
    else:
         print(res.status_code, "page not fount "+str(i))
         break      

200 page not fount 6

In [0]:
exclude_list = ['313','315','319','333','339','344','679','1066','1336']
df2=spark.createDataFrame(df)
df1 = df2[~df2['patient_id'].isin(exclude_list)]

In [0]:
#User defined function to change the date time to required format
dateudf=F.udf(lambda x: pd.to_datetime(x,unit='ms'),TimestampType())

In [0]:
#Changing the date format of appointment_date and adding a column with adls path where audio files will be stored#r
df1=df1.withColumn('appointment_date', dateudf(F.col("appointment_date")))

In [0]:
#creating temp view to store the data coming from the API
df1.createOrReplaceTempView('updates')

In [0]:
# %sql
# --creating table cyb.silver_data if it doesn't exist
# CREATE TABLE IF NOT EXISTS cyb.bronze_data(
#   index_id long ,
#   File_Arrival_Date DATE,
#   appointment_id long,
#   patient_id long,
#   patient_age long,
#   patient_gender string,
#   patient_type string,
#   appointment_date timestamp,
#   session_number long,
#   practitioner_id long,
#   practitioner_age long,
#   practitioner_gender string,
#   practitioner_experience string,
#   fees long ,
#   social_wellbeing long,
#   work_wellbeing long,
#   goals_performance long,
#   empathy_performance long,
#   personal_wellbeing long,
#   family_wellbeing long,
#   confidence_performance long,
#   primary_problem_area string,
#   secondary_problem_area string,   
#   medicare_rebate long,
#   audio_record_file_url string,
#   source string,
#   source string,
#   )
# USING delta
# PARTITIONED BY (patient_id)
# LOCATION 'dbfs:/mnt/cyberclinic/TableData/bronze_data_1'

In [0]:
%sql
delete from cyb.bronze_data where source = 'API';
INSERT into cyb.bronze_data(
        index_id,
        File_Arrival_Date,
        appointment_id,
        patient_id,
        patient_age,
        patient_gender,
        patient_type,
        appointment_date,
        session_number,
        practitioner_id,
        practitioner_age,
        practitioner_gender,
        practitioner_experience,
        fees,
        social_wellbeing,
        work_wellbeing,
        goals_performance,
        empathy_performance,
        personal_wellbeing,
        family_wellbeing,
        confidence_performance,
        primary_problem_area,
        secondary_problem_area,   
        medicare_rebate,
        audio_record_file_url,
        source
        ) 
        select abs(hash(appointment_date ||  patient_id || session_number)) index_id,
              current_date(),
              appointment_id,
              patient_id,
              patient_age,
              patient_gender,
              patient_type,
              appointment_date,
              session_number,
              practitioner_id,
              practitioner_age,
              practitioner_gender,
              practitioner_experience,
              fees,
              social_wellbeing,
              work_wellbeing,
              goals_performance,
              empathy_performance,
              personal_wellbeing,
              family_wellbeing,
              confidence_performance,
              primary_problem_area,
              secondary_problem_area,   
              medicare_rebate,
              audio_record_file_url,
              'API'
            from updates;
            

num_affected_rows,num_inserted_rows
456,456


In [0]:
%sql
SELECT COUNT(*) FROM cyb.bronze_data;

count(1)
571
